In [1]:
from bs4 import BeautifulSoup
from requests import get

In [2]:
#Let website think that we are actual user
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [43]:
#Sample rentals.ca website
rentals = "https://rentals.ca/toronto?p=2"

In [44]:
response = get(rentals, headers=headers)
#200 is a good response
print(response)

<Response [200]>


In [45]:
print(response.text[:1000])

<!doctype html>





<html lang="en">

<head>
  <meta charset="utf-8">

  

<link rel="preconnect" href="https://use.typekit.net">
<link rel="preconnect" href="https://images.rentals.ca">
<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://maps.googleapis.com">
<link rel="preconnect" href="https://maps.gstatic.com">
<link rel="preconnect" href="https://www.google-analytics.com">
<link rel="preconnect" href="https://s7.addthis.com">
<link rel="preconnect" href="https://cdnjs.cloudflare.com">
<link rel="preconnect" href="https://static.hotjar.com">
<link rel="preconnect" href="https://script.hotjar.com">
<link rel="preconnect" href="https://vars.hotjar.com">
<link rel="preconnect" href="https://cdn.sift.com">
<link rel="preconnect" href="https://hexagon-analytics.com">
<link rel="preconnect" href="https://analytics.trovit.com">

<link rel="preload" as="script" href="/static/rentalsca-phoenix/js/app.a5cc35a02ae9.js">


  <meta http-equiv="x-ua-

In [46]:
html_soup = BeautifulSoup(response.text, 'html.parser')

all_script = html_soup.find_all('script')

json_list = []
for item in all_script:
    if str(item).startswith('''<script type="applic'''):
        json_list.append(str(item))

In [47]:
dic = {'name':[], 'price': [], 'latitude': [], 'longitude': [], 'url': [], 'location': [], 'room_type': []}
for item in json_list:
    i = 1
    j = 1
    temp_room_type = []
    temp_price = []
    for words in item.split('\n'):
        if words.strip()[:6] == '''"name"''':
            if i == 1:
                dic['name'].append(words.strip()[7:-1].strip().strip('\"'))
                i += 1
            elif i == 2:
                dic['location'].append(''.join(e for e in words.strip()[7:-1] if e.isalnum()))
                i += 1
            else:
                temp_room_type.append(words.strip()[7:-1].strip().strip('\"'))
        if words.strip()[:7] == '''"price"''':
            temp_price.append(float(words.strip()[8:].strip()[:-1]))
        if words.strip()[:10] == '''"latitude"''':
            dic['latitude'].append(float(words.strip()[11:].strip()[:-1]))
        if words.strip()[:11] == '''"longitude"''':
            dic['longitude'].append(float(words.strip()[12:].strip()[:-1]))
        if words.strip()[:5] == '''"url"''' and j == 1:
            dic['url'].append(words.strip()[6:-1].strip().strip('\"'))
            j += 1
    if len(temp_room_type) == 0 and len(temp_price) == 0:
        if len(dic['name']) != 0:
            dic['name'].pop()
    else:
        dic['room_type'].append(temp_room_type)
        dic['price'].append(temp_price)

In [48]:
dic

{'name': ['4001 Bayview Avenue - The Gates of Bayview',
  '18 BROWNLOW',
  '650 Parliament Street',
  '571 Prince Edward Dr N',
  '1072 Lansdowne Ave',
  '59 E Liberty St',
  '17 Powerhouse St',
  'Queen &amp; Broadview 2Bed 2Bath Apartment',
  'Panorama Apartments',
  '2 Glenhaven Street'],
 'price': [[2182.0, 2927.0, 3587.0],
  [2250.0, 2350.0],
  [1400.0, 1550.0, 1700.0, 1999.0],
  [2000.0, 2300.0, 2700.0, 3850.0],
  [1495.0],
  [1950.0],
  [1800.0],
  [3200.0],
  [2100.0],
  [1912.0]],
 'latitude': [43.803980200030715,
  43.7064477,
  43.6697615,
  43.6602184,
  43.6693385,
  43.6383856,
  43.6697085,
  43.6595219,
  43.74717,
  43.691757],
 'longitude': [-79.392614364624,
  -79.391297,
  -79.370402,
  -79.510063,
  -79.447671,
  -79.413543,
  -79.447829,
  -79.3476295,
  -79.5819,
  -79.469902],
 'url': ['https://rentals.ca/north-york/4001-bayview-avenue-the-gates-of-bayview',
  'https://rentals.ca/toronto/18-brownlow-ave-1',
  'https://rentals.ca/toronto/650-parliament-street',
 

In [54]:
def scrapping_rentals_ca(pagenum = 50):
    dic = {'name':[], 'price': [], 'latitude': [], 'longitude': [], 'url': [], 'location': [], 'room_type': []}
    initial_link = 'https://rentals.ca/toronto?p='
    for i in range(pagenum):
        link = initial_link + str(i+1)
        response = get(link, headers=headers)
        html_soup = BeautifulSoup(response.text, 'html.parser')

        all_script = html_soup.find_all('script')

        json_list = []
        for item in all_script:
            if str(item).startswith('''<script type="applic'''):
                json_list.append(str(item))
        
        for item in json_list:
            i = 1
            j = 1
            temp_room_type = []
            temp_price = []
            for words in item.split('\n'):
                if words.strip()[:6] == '''"name"''':
                    if i == 1:
                        dic['name'].append(words.strip()[7:-1].strip().strip('\"'))
                        i += 1
                    elif i == 2:
                        dic['location'].append(''.join(e for e in words.strip()[7:-1] if e.isalnum()))
                        i += 1
                    else:
                        temp_room_type.append(words.strip()[7:-1].strip().strip('\"'))
                if words.strip()[:7] == '''"price"''':
                    temp_price.append(float(words.strip()[8:].strip()[:-1]))
                if words.strip()[:10] == '''"latitude"''':
                    dic['latitude'].append(float(words.strip()[11:].strip()[:-1]))
                if words.strip()[:11] == '''"longitude"''':
                    dic['longitude'].append(float(words.strip()[12:].strip()[:-1]))
                if words.strip()[:5] == '''"url"''' and j == 1:
                    dic['url'].append(words.strip()[6:-1].strip().strip('\"'))
                    j += 1
            if len(temp_room_type) == 0 and len(temp_price) == 0:
                if len(dic['name']) != 0:
                    dic['name'].pop()
            else:
                dic['room_type'].append(temp_room_type)
                dic['price'].append(temp_price)
    return dic

In [55]:
scrapped = scrapping_rentals_ca()

In [56]:
scrapped

{'name': ['120 Raglan Avenue',
  '310-312 Lonsdale Road',
  '35 Walmer Road',
  '50 Spadina Road',
  '45 Carlton Street',
  '191 Sherbourne Street',
  '201 Sherbourne Street',
  'Goldengate Apartments',
  '730 Dovercourt Road - Doversquare Apartments',
  '4001 Bayview Avenue - The Gates of Bayview',
  '18 BROWNLOW',
  '650 Parliament Street',
  '571 Prince Edward Dr N',
  '1072 Lansdowne Ave',
  '59 E Liberty St',
  '17 Powerhouse St',
  'Queen &amp; Broadview 2Bed 2Bath Apartment',
  'Panorama Apartments',
  '3018 Yonge Street',
  '2 Grosvenor Street',
  '5600 Sheppard Avenue East',
  '2801 Jane St.',
  '806 Kennedy Road',
  'Meridian I - 15 Greenview #1047',
  '82 Warren Road',
  'Heathdale Court',
  'Tobermory Terrace',
  'Residences at 77',
  '125 Bamburgh Circle',
  'The Selby',
  'Islington Terrace #3532',
  '1650 Bathurst Street',
  '1065 Eglinton Ave West',
  '3575 Bathurst Street',
  '2701 Eglinton Avenue West',
  '130 Raglan Avenue',
  '580 The East Mall',
  '778 Broadview Av

In [60]:
import csv
csv_columns = list(k for k in dic.keys())
csv_file = "scrapping.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in scrapped:
            writer.writerow(data)
except IOError:
    print("I/O error")

AttributeError: 'str' object has no attribute 'keys'

In [58]:
csv_columns = list(k for k in dic.keys())

In [59]:
csv_columns

['name', 'price', 'latitude', 'longitude', 'url', 'location', 'room_type']

In [61]:
type(scrapped)

dict

In [62]:
for item in scrapped:
    print(item)

name
price
latitude
longitude
url
location
room_type


In [63]:
import pandas as pd

In [65]:
pd.DataFrame.from_dict(scrapped)

ValueError: arrays must all be same length

In [66]:
[len(x) for x in scrapped.values()]

[451, 500, 500, 500, 500, 500, 500]

In [68]:
[x for x in scrapped.keys()]

['name', 'price', 'latitude', 'longitude', 'url', 'location', 'room_type']